In [121]:
from kinematicsrobotics import kinematics
from kinematicsrobotics  import dataprocessing
from kinematicsrobotics  import plot
from kinematicsrobotics  import spacemapping
import pandas as pd
import numpy as np

In [122]:
# Modelagem da cinemática direta usando Denavit-hartenberg
Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0],
        ['theta_3',0,18,-180,0],
        ['theta_4',0,0,90,90],
        ['theta_5',18,0,0,0]
]

robo = kinematics.Robo("Robo",Elos)


In [108]:
pose = [0,0,0,0,0]
frame = robo.frame_effector(pose)
#np.array(robo.frame_effector(pose)[:3]).reshape(-1)
type(frame[0][0])



sympy.core.numbers.Float

In [88]:
# Converter valores simbólicos para floats
lista_float = [[float(valor) for valor in linha] for linha in frame]

# Exibir a lista de floats resultante
type(lista_float[0][0])

float

In [123]:
# Mapeamento do espaço das juntas e o espaço operacional
joins = [[0, 10],[0, 10],[0, 10],[0, 10],[0, 0]]
steps = [10, joins[1][1]//2, joins[2][1]//2, joins[3][1]//2, 1]

dataset = spacemapping.mapping(robo,joins,steps)

dataset.head(3)

,theta_1,theta_2,theta_3,theta_4,theta_5,R_11,R_12,R_13,p_x,R_21,R_22,R_23,p_y,R_31,R_32,R_33,p_z
0,0,0,0,0,0,6.12323399573677e-17,0,1.00000000000000,54.0000000000000,0,-1.00000000000000,0,0,1.00000000000000,0,-6.12323399573677e-17,10.0000000000000
1,0,0,0,5,0,-0.0871557427476581,0,0.996194698091746,53.9315045656514,0,-1.00000000000000,0,0,0.996194698091746,0,0.0871557427476581,11.5688033694578
2,0,0,0,10,0,-0.173648177666930,0,0.984807753012208,53.7265395542197,0,-1.00000000000000,0,0,0.984807753012208,0,0.173648177666930,13.1256671980047


In [114]:
print(type(dataset['p_x'].loc[2]))

<class 'sympy.core.numbers.Float'>


### Teste

In [23]:
path = r'C:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics\src\data\raw\dataset.csv'
df_dataset = pd.read_csv(path)
df_dataset.head(3)

,theta_1,theta_2,theta_3,theta_4,theta_5,R_11,R_12,R_13,p_x,R_21,R_22,R_23,p_y,R_31,R_32,R_33,p_z
0,0,0,0,0,0,6.123234e-17,0.0,1.000000,54.000000,0.0,-1.0,0.0,0.0,1.000000,0,-6.123234e-17,10.00000
1,0,0,0,12,0,-2.079117e-01,0.0,0.978148,53.606657,0.0,-1.0,0.0,0.0,0.978148,0,2.079117e-01,13.74241
2,0,0,0,24,0,-4.067366e-01,0.0,0.913545,52.443818,0.0,-1.0,0.0,0.0,0.913545,0,4.067366e-01,17.32126


In [31]:
print(type(df_dataset['p_x'].loc[0]))

<class 'numpy.float64'>


## pre-processamento

In [115]:
## Altura negativa
height = 0
dataset = dataprocessing.remove_height(df_dataset,height)

## Elos negativos
Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0],
        ['theta_3',0,18,-180,0]
        ]

dataset = dataprocessing.remove_height_join(dataset,Elos,'bot',['theta_1','theta_2','theta_3'],height)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mathe\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mathe\AppData\Local\Temp\ipykernel_40780\1073924918.py", line 13, in <module>
    dataset = dataprocessing.remove_height_join(dataset,Elos,'bot',['theta_1','theta_2','theta_3'],height)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics\src\kinematicsrobotics\dataprocessing.py", line 22, in remove_height_join
    # Se altura alcançada em Z for menor que a desejada
        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics\src\kinematicsrobotics\forwardkinematics.py", line 137, in frame_effector
  F

In [124]:
# Usando 10 mm de raio
radius = 1
# Análise de vizinhos é no espaço operacional 
attr_neighbors = ['p_x','p_y','p_z']
# O critério de escolha é no espaço das juntas
attr_redundancy = ['theta_2','theta_3','theta_4','theta_5']

dataset_1mm = dataprocessing.remove_redundancy(dataset,radius,attr_neighbors, attr_redundancy)


TypeError: loop of ufunc does not support argument 0 of type Zero which has no callable sqrt method